# Introduction to Working with User Inputs

**Last Updated:** 2021-04-15

<details>
    <summary> Release Notes </summary>
    
    2021-04-15
        - Initial notebook release
    
</details>

In the previous modules, we have been working with scripts that can operate own their own once they have been run. However, once you start working with more advanced scripts, you may want to start building in ways to customize their behavior based on a user's input. In fact, you have likely already seen examples of this type of behavior (albeit in a different language) when running any of the command line commands (see Working with the Command Line). Because there is a wide variety of methods for working with user inputs, this module will be the first in a series covering this topic. In this module, we will introduce:

1. How to prompt a user for information while a script is running
2. How to use `sys.argv` to access command line arguments


## Table of Contents
1. [Using This Notebook](#Using-This-Notebook)
2. [Requesting User Input During Execution](#Requesting-User-Input-During-Execution)
3. [Simple Command Line Argument Parsing using `sys.argv`](#Simple-Command-Line-Argument-Parsing-using-sys.argv)
4. [Closing](#Closing)


## Using This Notebook

Unlike notebooks from previous modules, **you should not run all cells in this notebook**. Instead, step through each cell one at a time.

This notebook uses Jupyter Notebook magic functions to help simulate command line inputs. In particular, there are two magic functions that are essential to this module:

- `%writefile`: Saves the content of the code cell to the specified file name
- `%run `: Equivalent to calling `python3` in a terminal session
    - To run any scripts below on your local terminal after downloading them, you would use `python3 my_script.py`


## Requesting User Input During Execution

One aspect of working with user input is to prompt users *in the middle of running a script*. To showcase this functionality, let's first introduce a simple script that will print out an introduction containing your name and the number of pets you have (`greetings.py`).

In [ ]:
%%writefile greetings.py
#!usr/bin/bash python

my_name = "Anthony"
number_of_pets = 1

print("My name is {} and I have {} pet(s).".format(my_name, number_of_pets))

In [ ]:
%run greetings.py

While this script does the job admirably, what would happen if someone else wanted to use the script to generate their own introduction? Since we have coded in our name and number of pets, they would have to go in and change the script themselves. For a simple script like `greetings.py`, the change may not present an issue. However, you usually don't want others editing your code to personalize its behavior.

Instead, we make use of the `input([prompt])` function (See [documentation](https://docs.python.org/3/library/functions.html#input)). This function will print out the `prompt` to the command line and wait until the user types in the input. Once the user has finished typing in the input, it will return that input as a *string* and can be stored like any other variable. Let's use this function improve our script to prompt for a user's name and number of pets. If you run the cells below, you should be prompted for user input! **Note: this will overwrite the previous implementation of `greetings.py` from earlier.**

In [ ]:
%%writefile greetings.py
#!usr/bin/bash python

my_name = input("Name: ")
number_of_pets = input("Number of Pets: ")

print("My name is {} and I have {} pet(s).".format(my_name, number_of_pets))

In [ ]:
%run greetings.py

When creating the prompt, I normally include a trailing space character. This leaves a space between the prompt and the first character of the users input to make it easier to distinguish between the two. However, this is formatting is not required.

Next, let's create another script that will print out the sum of the two numbers that a user provides (`user_sum.py`).

In [ ]:
%%writefile user_sum.py
#!usr/bin/bash python

num_one = input("First Number: ")
num_two = input("Second Number: ")

print(num_one + num_two)

In [ ]:
%run user_sum.py

This implementation **immediately** introduces one of the most common sources of bugs when working with `input()`: the input is **always returned as a string**. In the example above, because we do not explicitly recast the inputs to a numeric type (e.g., integer), Python will treat the addition as *string addition* instead of *integer addition*. Let's modify the script so that we recast the inputs as integers prior to claculating the sum.

In [ ]:
%%writefile user_sum.py
#!usr/bin/bash python

num_one = int(input("First Number: "))
num_two = int(input("Second Number: "))

print(num_one + num_two)

In [ ]:
%run user_sum.py

While the input type problem may appear to be solved, rerun the `$run` cell above and pass in an input that *cannot be converted to an integer*, such as "3.2" or "1a". This can be an example of a user input error (e.g., accidently hitting the wrong key).

You should have caused Python to raise a `ValueError`: this means that Python could not cast the input to the request value type. While this may seem like a major issue, there is actually a simple solution for dealing with improper inputs by "catching" these exceptions using a `while-try-except` pattern. We can use this pattern to create a function that will safely prompt a user for an integer by "catching" the exception and requesting a new acceptable input.

In [ ]:
%%writefile user_sum.py
#!usr/bin/bash python

def prompt_for_integer(prompt):
    while True:
        try:
            return(int(input(prompt)))
        except ValueError:
            print("Please enter a valid integer.")

num_one = prompt_for_integer("First Number: ")
num_two = prompt_for_integer("Second Number: ")

print(num_one + num_two)

In [ ]:
%run user_sum.py

While error handling will be introduced comprehensively later, let's quickly dissect how the `while-try-except` pattern works. First, the outer `while True` loop will continue to run until the code within the loop either breaks out of the loop using a `break` statement or returns, as in this example. During every run through the while loop, the code will execute the `try-except` block. The code within the `try` block will be executed. If this causes an error or exception (a ValueError in our case), the code within the `except` block is run *instead of crashing the entire script*. In the example above, we will continually use `input()` to prompt the user for a valid integer. Since our `except` block doesn't break out of the while loop or return, it prints the error message and then presents the prompt again.

With this validating function in place, we now have all of the basic functionality we need to safely get a user prompt while executing a script. We have implemented a way for:

1. Prompting a user for a specific type of input
2. Ensuring that input conforms to the expected type
3. Reprompting the user if the input is not valid.

## Simple Command Line Argument Parsing using `sys.argv`

In the first section of this module, we introduced a method for accepting inputs *in the middle of executing a script*. However, many scripts accept inputs, or "command line arguments", when they initially called. Passing these command line arguments to a Python script follows the standard syntax for any command or language:

```
python3 my_script.py argument_1 argument_2
```

Once you pass these arguments to a script, there are many different ways we can access and use these inputs. Some methods provide the minimal amount of functionality while others provide entire frameworks for customized UI and input methods. The `sys` module is an essential Python module for interacting with the local operating system. For our purposes, we are only interested in one particular part of the `sys` module: the `sys.argv` list.  The `sys.argv` list contains all of the arguments that were passed when a Python script was called. This  Importantly, `sys.argv` is just a Python list, so all list methods can be applied! Let's use a simple script (`print_sys_argv.py`) to explore some of the details.

One thing to think about: what will be the type of the arguments?

In [ ]:
%%writefile print_sys_argv.py
#!usr/bin/bash python

import sys


# Print out the length of the list
print("Length of sys.argv: {}".format(len(sys.argv)))

# Print out the entire list
print(sys.argv)  

# Loop over each argument and print
for arg in sys.argv:
    print("{}: {}".format(arg, type(arg)))

This script will print out the entire list followed by each argument and their type. The cell below will call the script with three arguments: 1) a string-like argument, 2) a interger-like argument, and 3) a float-like argument.

In [ ]:
%run print_sys_argv.py argument_1 2 3.0

After running the cell above, take a look at the printed output. As expected, `sys.argv` is a Python list of all of the arguments. However, there it may seem like there is 1 extra argument: the name of the script. If you pay attention to the syntax of running a Python script, we are *actually* running `python3` and not the script itself, so `sys.argv` captures the script name as the "first" argument. Practiaclly, this just means:

1. We'll always know what script we initally ran (good for complicated cases)
2. The arguments list (minus the script name) is effectively 1-indexed (the 2nd argument is `sys.argv[2]`)

Next, notice that all of the arguments are initially parsed as *strings*, just like the `input()` method! As a result, you will still use similar type casting and valdiation approaches introduced earlier if you want to prompt the user for non-string inputs.

Let's now rewrite our `greetings.py` script to utilize command line arguments instead (saved as `greetings_cli.py`).

In [ ]:
%%writefile greetings_cli.py
#!usr/bin/bash python

import sys

my_name = sys.argv[1]
number_of_pets = sys.argv[2]

print("My name is {} and I have {} pet(s).".format(my_name, number_of_pets))

In [ ]:
%run greetings_cli.py Anthony 1

This is perfect! We can now prompt the user to input all of the information *at the beginning* instead of stopping to request input multiple times throughout the script! However, using `sys.argv` is one of the more limited methods for processing user inputs. For example, nothing prevents a user from inputing the inputs *in the wrong order or from passing any number of arguments*:

In [ ]:
%run greetings_cli.py 1 Anthony argument_3 3453 adsg

In fact, *unless a user opens up and inspects the script directly, there is no way of figuring out what the expected arguments are*. For simple cases like the one above or in early developement, this method may be all you need. For more complex or user-focused cases, we would need to start looking at more complex argument parsing modules, such as the `argparse` modules (See Introduction to Argparse: Pt. 1)

## Closing

In this module, we introduced the two basic methods for working with user input: requesting user input while running a script using the `input()` function and parsing command line arguments using the `sys.argv` list. As mentioned throughout the module, the are additional methods for creating more complex input parsing systems. However, the methods introduced in this module can serve almost all introductory needs.

In the next module, we introduce the `argparse` module, the built-in command line argument parsing module that is the next step up from `sys.argv`.